In [1]:
#imports
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

#pandas profiling
from pandas_profiling import ProfileReport

#sklearn stuff
from sklearn.pipeline import Pipeline
from feature_engine.discretisers import EqualFrequencyDiscretiser
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, SVC

In [2]:
# Import the Dataset
data = pd.read_csv('Credit_Data.csv')
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,female,university,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,default
1,2,120000,female,university,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,default
2,3,90000,female,university,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,not default
3,4,50000,female,university,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,not default
4,5,50000,male,university,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,not default


In [3]:
# Rename column
data['default'] = data['default payment next month']
# Drop unneeded columns
data = data.drop(['ID', 'default payment next month'], axis=1)

In [4]:
# Drop duplicate rows
data = data.drop_duplicates()

# Check for nulls
nulls = data.isnull().sum()
nulls

LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
default      0
dtype: int64

In [5]:
# Check datatypes
data.dtypes

LIMIT_BAL     int64
SEX          object
EDUCATION    object
MARRIAGE      int64
AGE           int64
PAY_0         int64
PAY_2         int64
PAY_3         int64
PAY_4         int64
PAY_5         int64
PAY_6         int64
BILL_AMT1     int64
BILL_AMT2     int64
BILL_AMT3     int64
BILL_AMT4     int64
BILL_AMT5     int64
BILL_AMT6     int64
PAY_AMT1      int64
PAY_AMT2      int64
PAY_AMT3      int64
PAY_AMT4      int64
PAY_AMT5      int64
PAY_AMT6      int64
default      object
dtype: object

In [6]:
# Encode as needed
le = LabelEncoder()
le.fit(data['SEX'])
data['SEX'] = le.transform(data['SEX'])

In [7]:
# Encode as needed
le = LabelEncoder()
le.fit(data['default'])
data['default'] = le.transform(data['default'])

In [8]:
# One-hot encode objects
data = pd.get_dummies(data)

In [9]:
# Check datatypes
data.dtypes

LIMIT_BAL                    int64
SEX                          int32
MARRIAGE                     int64
AGE                          int64
PAY_0                        int64
PAY_2                        int64
PAY_3                        int64
PAY_4                        int64
PAY_5                        int64
PAY_6                        int64
BILL_AMT1                    int64
BILL_AMT2                    int64
BILL_AMT3                    int64
BILL_AMT4                    int64
BILL_AMT5                    int64
BILL_AMT6                    int64
PAY_AMT1                     int64
PAY_AMT2                     int64
PAY_AMT3                     int64
PAY_AMT4                     int64
PAY_AMT5                     int64
PAY_AMT6                     int64
default                      int32
EDUCATION_graduate school    uint8
EDUCATION_high school        uint8
EDUCATION_other              uint8
EDUCATION_university         uint8
dtype: object

In [10]:
# View the makeup of the data
data.describe()

,LIMIT_BAL,SEX,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,EDUCATION_graduate school,EDUCATION_high school,EDUCATION_other,EDUCATION_university
count,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000,...,2.996500e+04,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000,29965.000000
mean,167442.005006,0.396262,1.551877,35.487969,-0.016753,-0.131854,-0.164392,-0.218922,-0.264509,-0.289438,...,5.927983e+03,5231.688837,4831.617454,4804.897047,5221.498014,0.778742,0.352511,0.164025,0.015618,0.467846
std,129760.135222,0.489128,0.521997,9.219459,1.123492,1.196322,1.195878,1.168175,1.132220,1.149090,...,2.305346e+04,17616.361124,15674.464538,15286.372298,17786.976864,0.415101,0.477760,0.370304,0.123995,0.498973
min,10000.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,0.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,8.500000e+02,390.000000,300.000000,261.000000,131.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,140000.000000,0.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.010000e+03,1804.000000,1500.000000,1500.000000,1500.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,240000.000000,1.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000e+03,4512.000000,4016.000000,4042.000000,4000.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,1000000.000000,1.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,1.684259e+06,896040.000000,621000.000000,426529.000000,528666.000000,1.000000,1.000000,1.000000,1.000000,1.000000


There are 29,965 distinct customer loans
The average credit limit is 167,442, minimum credit limit is 10,000 and max is 1,000,000
Education level is mostly graduate school and university
Average age is 35.5 with a standard deviation of 9.2
There are 22% of customer loans that will default next month

In [12]:
# Shift pay features for -2
data['PAY_0'] = data['PAY_0'].replace(-2,0)
data['PAY_2'] = data['PAY_2'].replace(-2,0)
data['PAY_3'] = data['PAY_3'].replace(-2,0)
data['PAY_4'] = data['PAY_4'].replace(-2,0)
data['PAY_5'] = data['PAY_5'].replace(-2,0)
data['PAY_6'] = data['PAY_6'].replace(-2,0)

# Shift pay features for -1
data['PAY_0'] = data['PAY_0'].replace(-1,0)
data['PAY_2'] = data['PAY_2'].replace(-1,0)
data['PAY_3'] = data['PAY_3'].replace(-1,0)
data['PAY_4'] = data['PAY_4'].replace(-1,0)
data['PAY_5'] = data['PAY_5'].replace(-1,0)
data['PAY_6'] = data['PAY_6'].replace(-1,0)

In [13]:
data.head()

,LIMIT_BAL,SEX,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,EDUCATION_graduate school,EDUCATION_high school,EDUCATION_other,EDUCATION_university
0,20000,0,1,24,2,2,0,0,0,0,...,689,0,0,0,0,0,0,0,0,1
1,120000,0,2,26,0,2,0,0,0,2,...,1000,1000,1000,0,2000,0,0,0,0,1
2,90000,0,2,34,0,0,0,0,0,0,...,1500,1000,1000,1000,5000,1,0,0,0,1
3,50000,0,1,37,0,0,0,0,0,0,...,2019,1200,1100,1069,1000,1,0,0,0,1
4,50000,1,1,57,0,0,0,0,0,0,...,36681,10000,9000,689,679,1,0,0,0,1


In [14]:
# Check the correlations of the variables
data.corr()

,LIMIT_BAL,SEX,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,EDUCATION_graduate school,EDUCATION_high school,EDUCATION_other,EDUCATION_university
LIMIT_BAL,1.000000,-0.025015,-0.108081,0.144643,-0.170975,-0.197039,-0.191185,-0.180492,-0.169903,-0.167674,...,0.178584,0.210375,0.203451,0.217421,0.219807,0.153871,0.258512,-0.139583,0.013468,-0.147280
SEX,-0.025015,1.000000,0.031372,0.091057,0.034612,0.044685,0.041064,0.039565,0.038349,0.031869,...,0.001390,0.008599,0.002227,0.001665,0.002765,-0.039742,0.022890,0.007807,-0.008502,-0.025598
MARRIAGE,-0.108081,0.031372,1.000000,-0.413901,-0.011770,-0.010001,0.000565,-0.004201,-0.002941,0.001885,...,-0.008099,-0.003545,-0.012668,-0.001207,-0.006646,0.024019,0.142187,-0.110925,-0.008393,-0.051735
AGE,0.144643,0.091057,-0.413901,1.000000,-0.001401,-0.008783,-0.014171,-0.007167,-0.013141,-0.016816,...,0.021726,0.029181,0.021306,0.022777,0.019409,-0.013619,-0.100403,0.231322,0.008952,-0.077762
PAY_0,-0.170975,0.034612,-0.011770,-0.001401,1.000000,0.698771,0.517209,0.460391,0.424593,0.373883,...,-0.057092,-0.062507,-0.061768,-0.053659,-0.048616,-0.396200,-0.068575,0.042082,-0.025403,0.040742
PAY_2,-0.197039,0.044685,-0.010001,-0.008783,0.698771,1.000000,0.663434,0.512599,0.462524,0.406853,...,-0.054946,-0.059989,-0.054108,-0.048843,-0.043218,-0.327095,-0.083326,0.043769,-0.038228,0.056801
PAY_3,-0.191185,0.041064,0.000565,-0.014171,0.517209,0.663434,1.000000,0.678639,0.551006,0.492344,...,-0.073776,-0.057812,-0.053961,-0.050916,-0.048547,-0.286898,-0.071400,0.041765,-0.041327,0.047639
PAY_4,-0.180492,0.039565,-0.004201,-0.007167,0.460391,0.512599,0.678639,1.000000,0.745171,0.602485,...,-0.036951,-0.074166,-0.055400,-0.052827,-0.048917,-0.268937,-0.067921,0.039347,-0.036484,0.044898
PAY_5,-0.169903,0.038349,-0.002941,-0.013141,0.424593,0.462524,0.551006,0.745171,1.000000,0.740088,...,-0.039651,-0.037341,-0.066032,-0.051898,-0.046185,-0.260721,-0.057033,0.029220,-0.035144,0.041656
PAY_6,-0.167674,0.031869,0.001885,-0.016816,0.373883,0.406853,0.492344,0.602485,0.740088,1.000000,...,-0.039936,-0.040236,-0.030532,-0.066969,-0.045436,-0.244290,-0.046825,0.020000,-0.034644,0.038601


The repayment status variables PAY_0 - PAY_6 have the highest correlation in the dataset with customer defaults.

In [15]:
# Rename column PAY_0 to PAY_1
data = data.rename(columns={'PAY_0': 'PAY_1'})
data.head()

,LIMIT_BAL,SEX,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,EDUCATION_graduate school,EDUCATION_high school,EDUCATION_other,EDUCATION_university
0,20000,0,1,24,2,2,0,0,0,0,...,689,0,0,0,0,0,0,0,0,1
1,120000,0,2,26,0,2,0,0,0,2,...,1000,1000,1000,0,2000,0,0,0,0,1
2,90000,0,2,34,0,0,0,0,0,0,...,1500,1000,1000,1000,5000,1,0,0,0,1
3,50000,0,1,37,0,0,0,0,0,0,...,2019,1200,1100,1069,1000,1,0,0,0,1
4,50000,1,1,57,0,0,0,0,0,0,...,36681,10000,9000,689,679,1,0,0,0,1


In [16]:
# Rename education columns
data = data.rename(columns={'EDUCATION_graduate school':'Grad School', 'EDUCATION_high school':'High School',
                           'EDUCATION_other':'Edu_Other', 'EDUCATION_university':'University'})
data.head()

,LIMIT_BAL,SEX,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,Grad School,High School,Edu_Other,University
0,20000,0,1,24,2,2,0,0,0,0,...,689,0,0,0,0,0,0,0,0,1
1,120000,0,2,26,0,2,0,0,0,2,...,1000,1000,1000,0,2000,0,0,0,0,1
2,90000,0,2,34,0,0,0,0,0,0,...,1500,1000,1000,1000,5000,1,0,0,0,1
3,50000,0,1,37,0,0,0,0,0,0,...,2019,1200,1100,1069,1000,1,0,0,0,1
4,50000,1,1,57,0,0,0,0,0,0,...,36681,10000,9000,689,679,1,0,0,0,1


In [17]:
# Fix values for the variable SEX (1=male; 2=female)
data['SEX'] = data['SEX'].replace(0,2)
data.head()

,LIMIT_BAL,SEX,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,Grad School,High School,Edu_Other,University
0,20000,2,1,24,2,2,0,0,0,0,...,689,0,0,0,0,0,0,0,0,1
1,120000,2,2,26,0,2,0,0,0,2,...,1000,1000,1000,0,2000,0,0,0,0,1
2,90000,2,2,34,0,0,0,0,0,0,...,1500,1000,1000,1000,5000,1,0,0,0,1
3,50000,2,1,37,0,0,0,0,0,0,...,2019,1200,1100,1069,1000,1,0,0,0,1
4,50000,1,1,57,0,0,0,0,0,0,...,36681,10000,9000,689,679,1,0,0,0,1


In [18]:
# Fix values for the variable default (Yes=1; No=0)
data['default'] = data['default'].replace(1,2)

In [19]:
data['default'] = data['default'].replace(0,1)

In [20]:
data['default'] = data['default'].replace(2,0)
data.head()

,LIMIT_BAL,SEX,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,Grad School,High School,Edu_Other,University
0,20000,2,1,24,2,2,0,0,0,0,...,689,0,0,0,0,1,0,0,0,1
1,120000,2,2,26,0,2,0,0,0,2,...,1000,1000,1000,0,2000,1,0,0,0,1
2,90000,2,2,34,0,0,0,0,0,0,...,1500,1000,1000,1000,5000,0,0,0,0,1
3,50000,2,1,37,0,0,0,0,0,0,...,2019,1200,1100,1069,1000,0,0,0,0,1
4,50000,1,1,57,0,0,0,0,0,0,...,36681,10000,9000,689,679,0,0,0,0,1


All datatypes have been changed to numeric values, duplicate and unwanted rows have been removed, education level has been split into separate variables, all value labels for SEX and default have been changed to match the Data Set and Attribute Information Source provided by I-Cheng Yeh, all -2 and -1 value labels for PAY_1 - PAY_6 have been grouped with the 0 value label to represent a duly paid status.

In [21]:
# Save the cleaned dataset in a csv file
data.to_csv('Cleaned_Credit.csv', index=False)